# Implementation of different versions of the benefit difference fairness function

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import multiprocessing as mp
from src.util import sigmoid
from src.feature_map import IdentityFeatureMap
from src.functions import cost_utility, demographic_parity
from src.plotting import plot_mean_over_lambdas, plot_median_over_lambdas
from src.training import train_multiple
from src.distribution import SplitDistribution

## The parameters used by the the original authors  

In [2]:
bias = True
dim_x = 1
dim_theta = dim_x + 1 if bias else dim_x

lambdas = np.logspace(-1, 2, base=10, endpoint=True, num=100)
lambdas = np.insert(arr=lambdas, obj=0, values=[0.0])

iterations=30

def util_func(**util_params):
    util = cost_utility(cost_factor=0.55, **util_params)
    return util

training_parameters = {    
    'save_path': "/home/fkretschmar/Documents/master-thesis/res/exp-004/",
    'model':{
        'theta': [-3.5, 0.6],
        'benefit_value_function': demographic_parity,
        'utility_value_function': util_func,
        'feature_map': IdentityFeatureMap(dim_theta),
        'keep_collected_data': False,
        'use_sensitve_attributes': False,
        'bias': bias
    },
    'optimization': {
        'time_steps':200,
        'epochs': 32,
        'batch_size':512,
        'learning_rate': 0.5,
        'decay_rate': 0.8,
        'decay_step': 30,
        'fairness_rates': lambdas,
        'test_at_every_timestep': False
    },
    'data': {
        'distribution': SplitDistribution(bias=bias),
        'keep_data_across_lambdas': True,
        'fraction_protected':0.5,
        'num_test_samples': 20480,
        'num_decisions': 32 * 512
    }
}


## Correct Benefit Difference

In [3]:
def fairness_function(**fairness_kwargs):
    policy = fairness_kwargs["policy"]
    x = fairness_kwargs["x"]
    s = fairness_kwargs["s"]
    y = fairness_kwargs["y"]
    decisions = fairness_kwargs["decisions"]
    ips_weights = fairness_kwargs["ips_weights"]

    benefit = policy.benefit_function(decisions=decisions, y=y)

    if ips_weights is not None:
        benefit *= ips_weights

    log_gradient = policy._log_gradient(x, s)
    benefit_grad = benefit * log_gradient
        
    # benefit-difference * grad-benefit-difference
    return policy._mean_difference(benefit, s) * policy._mean_difference(benefit_grad, s)

training_parameters["model"]['fairness_function'] = fairness_function

statistics = train_multiple(training_parameters, iterations=iterations, verbose=True, asynchronous=True)

plot_mean_over_lambdas(statistics, "results_mean.png")
plot_median_over_lambdas(statistics, "results_median.png")

ntial_learning.py", line 95, in consequential_learning
    pi.update(x, s, y, learning_rate, training_args["optimization"]["batch_size"], training_args["optimization"]["epochs"])
  File "/home/fkretschmar/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/fkretschmar/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/fkretschmar/Documents/master-thesis/src/training.py", line 128, in _train_single
    for u, bd, pi in consequential_learning(**training_parameters):
  File "/home/fkretschmar/Documents/master-thesis/src/policy.py", line 257, in update
    gradient = self._policy_gradient(X_batch, S_batch, Y_batch, ips_weights_batch)
  File "/home/fkretschmar/Documents/master-thesis/src/training.py", line 128, in _train_single
    for u, bd, pi in consequential_learning(**training_parameters):
  File "/home/fkretschmar/Documents/master-thesis/src/training.py", line 128, in _train_

KeyboardInterrupt: 